![Coleridge Initiative Logo](images/CI_horizontal.png)

<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Brian Kim, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Avishek Kumar, Jonathan Morgan, Ekaterina Levitskaya.

# API Calls with Python

APIs (application programming interfaces) are hosted on web servers. When you type www.google.com in your browser's address bar, your computer is actually asking the www.google.com server for a webpage, which it then returns to your browser. APIs work much the same way, except instead of your web browser asking for a webpage, your program asks for data. This data is usually returned in JSON format. 

To retrieve data, we make a request to a webserver. The server then replies with our data. In Python, we'll use the `requests` library to do this.

### Python Setup

In [57]:
# interacting with websites and web-APIs
import requests 

# data manipulation
import pandas as pd 

# normalize nested JSON files
from pandas.io.json import json_normalize

## How does the request package work?

We first need to understand what information can be accessed from the API. We use an example of the **PatentsView API** (www.patentsview.org) to make the API call and check the information we get. 

### About PatentsView API

The PatentsView platform is built on data derived from the US Patent and Trademark Office (USPTO) to link inventors, their organizations, locations, and overall patenting activity. The PatentsView API provides programmatic access to longitudinal data and metadata on patents, inventors, companies, and geographic locations. 

To access the API, we use the `request` function. In oder to tell Python what to access we need to specify the url of the API endpoint.

PatentsView has several API endpoints. An endpoint is a server route that is used to retrieve different data from the API. You can think of the endpoints as just specifying what types of data you want.Examples of PatentsView API endpoints: http://www.patentsview.org/api/doc.html

Many times, we need to request a key from the data provider in order to access an API. For example, if you wanted to access the Twitter API, then you would need to get a Twitter developer account and access token (see https://developer.twitter.com/en/docs/basics/authentication/overview/oauth).
Currently no key is necessary to access the PatentsView API. 

### Making a Request
When you ping a website or portal for information this is called making a request. That is exactly what the `requests` library has been designed to do. However, we need to provide a query URL according to the format defined by PatentsView. The details on how to do that is explained at this link: https://www.patentsview.org/api/query-language.html.

Following the directions detailed in the link above, let's build our first query URL.

**Query String Format**

The query string is always a single JSON object: **{`<field>`:`<value>`}**, where `<field>` is the name of a database field and `<value>` is the value the field will be compared to for equality (Each API Endpoint section contains a list of the data fields that can be selected for inclusion in output datasets).

We use the following base URL for the Patents Endpoint:

**Base URL**: `http://www.patentsview.org/api/patents/query?q={criteria}`



## Task example: Pull patents for Stanford University

In this example, we will only pull patents from one organization: Stanford University. Let's go to the Patents Endpoint (http://www.patentsview.org/api/patent.html) and find the appropriate field for the organization's name.

The variable that we need is called `"assignee_organization"` (organization name, if assignee is organization)

> _Note_: **Assignee**: the name of the entity - company, foundation, partnership, holding company or individual - that owns the patent. In this example we are looking at universities (organization-level).

We will pull from the API using a step-by-step process:
- build the query,
- get the response,
- check the response code,
- get the content,
- convert to table.

By the end, we should have data about patents that we can work with using the tools we've already learned.

### Build the URL query 

Let's build our first URL query by combining the base url with one criterion (name of the `assignee_organization`)

**base url**: `http://www.patentsview.org/api/patents/query?` + **criterion**: `q={"assignee_organization":stanford university"}`

In [58]:
# Save the URL as a variable.

base_url = 'http://www.patentsview.org/api/patents/query?'

In [59]:
base_url

'http://www.patentsview.org/api/patents/query?'

In [60]:
# Save our criterion as a variable.

criterion = 'q={"assignee_organization":"stanford university"}'

In [61]:
criterion

'q={"assignee_organization":"stanford university"}'

In [62]:
# Combine the base URL with our criterion to create a finalized URL link.

url = base_url + criterion

In [63]:
url

'http://www.patentsview.org/api/patents/query?q={"assignee_organization":"stanford university"}'

### Get the response

Now let's get the response using the URL defined above and the `requests` library.

In [64]:
# Get response from the URL.

r = requests.get(url)

### Check the response code

Before you can do anything with a website or URL in Python, it’s a good idea to check the current status code of said portal.

The following are the response codes for the PatentsView API:

`200` - the query parameters are all valid; the results will be in the body of the response.

`400` - the query parameters are not valid, typically either because they are not in valid JSON format, or a specified field or value is not valid; the “status reason” in the header will contain the error message.

`500` -  there is an internal error with the processing of the query; the “status reason” in the header will contain the error message.

Let's check the status of our response.

In [65]:
r.status_code  # Check the status code

200

We are good to go. Now let's get the content.

### Get the content
After a web server returns a response, you can collect the content you need by converting it into a JSON format.

JSON is a way to encode data structures like lists and dictionaries to strings that ensures that they are easily readable by machines. JSON is the primary format in which data is passed back and forth to APIs, and most API servers will send their responses in JSON format.

In [66]:
response = r.json()  # Convert response to JSON format

By default, we get information on `patent_id`, `patent_number`, and `patent_title`. At the end of the JSON you will see how many results are returned (variable `count`) and the total number of patents found (variable `total_patent_count`).

In [67]:
response  # View JSON

{'patents': [{'patent_id': '4200770',
   'patent_number': '4200770',
   'patent_title': 'Cryptographic apparatus and method'},
  {'patent_id': '4214918',
   'patent_number': '4214918',
   'patent_title': 'Method of forming polycrystalline semiconductor interconnections, resistors and contacts by applying radiation beam'},
  {'patent_id': '4233671',
   'patent_number': '4233671',
   'patent_title': 'Read only memory and integrated circuit and method of programming by laser means'},
  {'patent_id': '4234352',
   'patent_number': '4234352',
   'patent_title': 'Thermophotovoltaic converter and cell for use therein'},
  {'patent_id': '4325611',
   'patent_number': '4325611',
   'patent_title': 'Electrochromic material and electro-optical display using same'},
  {'patent_id': '4379697',
   'patent_number': '4379697',
   'patent_title': 'Stimulator array'},
  {'patent_id': '4380072',
   'patent_number': '4380072',
   'patent_title': 'XUV Laser and method'},
  {'patent_id': '4404068',
   'pate

JSON file has a dictionary structure (dictionaries are identified by curly backets `{}`), which includes keys and values (e.g. in `patent_id`:`4200770`, `patent_id` is a key and `4200770` is a value of that key).

In [68]:
type(response)  # Check the type of response variable

dict

We can check how many keys this JSON has by using `.keys()` function:

In [69]:
response.keys()

dict_keys(['patents', 'count', 'total_patent_count'])

Let's check the values of each key:

In [70]:
response['count']

25

In [71]:
response['total_patent_count']

143

In [72]:
response['patents']

[{'patent_id': '4200770',
  'patent_number': '4200770',
  'patent_title': 'Cryptographic apparatus and method'},
 {'patent_id': '4214918',
  'patent_number': '4214918',
  'patent_title': 'Method of forming polycrystalline semiconductor interconnections, resistors and contacts by applying radiation beam'},
 {'patent_id': '4233671',
  'patent_number': '4233671',
  'patent_title': 'Read only memory and integrated circuit and method of programming by laser means'},
 {'patent_id': '4234352',
  'patent_number': '4234352',
  'patent_title': 'Thermophotovoltaic converter and cell for use therein'},
 {'patent_id': '4325611',
  'patent_number': '4325611',
  'patent_title': 'Electrochromic material and electro-optical display using same'},
 {'patent_id': '4379697',
  'patent_number': '4379697',
  'patent_title': 'Stimulator array'},
 {'patent_id': '4380072',
  'patent_number': '4380072',
  'patent_title': 'XUV Laser and method'},
 {'patent_id': '4404068',
  'patent_number': '4404068',
  'patent_t

The `patents` key is interesting. It contains not just a single value, but a list of values (lists are identified by square brackets `[]`).

Dictionaries also can be hierarchical, or nested: the level `patents` contains a list (lists are identified by square brackets `[]`) with multiple elements in it.

In [73]:
type(response['patents'])  

list

In [74]:
len(response['patents'])  # Check how many elements the first level contains

25

In [75]:
response['patents']

[{'patent_id': '4200770',
  'patent_number': '4200770',
  'patent_title': 'Cryptographic apparatus and method'},
 {'patent_id': '4214918',
  'patent_number': '4214918',
  'patent_title': 'Method of forming polycrystalline semiconductor interconnections, resistors and contacts by applying radiation beam'},
 {'patent_id': '4233671',
  'patent_number': '4233671',
  'patent_title': 'Read only memory and integrated circuit and method of programming by laser means'},
 {'patent_id': '4234352',
  'patent_number': '4234352',
  'patent_title': 'Thermophotovoltaic converter and cell for use therein'},
 {'patent_id': '4325611',
  'patent_number': '4325611',
  'patent_title': 'Electrochromic material and electro-optical display using same'},
 {'patent_id': '4379697',
  'patent_number': '4379697',
  'patent_title': 'Stimulator array'},
 {'patent_id': '4380072',
  'patent_number': '4380072',
  'patent_title': 'XUV Laser and method'},
 {'patent_id': '4404068',
  'patent_number': '4404068',
  'patent_t

The key `patents` contains 25 elements, or, as we can see, mini-dictionaries (identified by the curly brackets `{}`), with their own keys and values. Every element in a list is a patent with its ID, number, and title.

Every patent's information can be accessed by calling:

In [76]:
response['patents'][0]  # Show the first patent only

{'patent_id': '4200770',
 'patent_number': '4200770',
 'patent_title': 'Cryptographic apparatus and method'}

In [77]:
response['patents'][1]  # Show the second patent only

{'patent_id': '4214918',
 'patent_number': '4214918',
 'patent_title': 'Method of forming polycrystalline semiconductor interconnections, resistors and contacts by applying radiation beam'}

### Convert to a dataframe

We can convert JSON information on patents into a table format very easily. 

In [78]:
df = pd.DataFrame(response['patents'])  # Convert to pandas dataframe
df

,patent_id,patent_number,patent_title
0,4200770,4200770,Cryptographic apparatus and method
1,4214918,4214918,Method of forming polycrystalline semiconducto...
2,4233671,4233671,Read only memory and integrated circuit and me...
3,4234352,4234352,Thermophotovoltaic converter and cell for use ...
4,4325611,4325611,Electrochromic material and electro-optical di...
5,4379697,4379697,Stimulator array
6,4380072,4380072,XUV Laser and method
7,4404068,4404068,Solid state method for synthesis reactions
8,4442206,4442206,"Method of using isotropic, porous-wall polymer..."
9,4471647,4471647,Gas chromatography system and detector and method


### <span style="color:red">Checkpoint 1: Pull patent data for another university</span>

Now try pulling patent data for another university (e.g. Georgetown University):
- build a query URL;
- make a request;
- get the response in JSON format;
- note the total number of patents;
- convert the JSON to a dataframe.

In [79]:
criterion_harvard = 'q={"assignee_organization":"harvard university"}'
url_harvard = base_url + criterion_harvard
r_harvard = requests.get(url_harvard)

In [80]:
r_harvard.status_code

200

In [81]:
response_harvard = r_harvard.json()  # Convert response to JSON format

In [82]:
# number patents
len(response_harvard['patents'])

10

In [83]:
df_harvard = pd.DataFrame(response_harvard['patents'])  # Convert to pandas dataframe
df_harvard

,patent_id,patent_number,patent_title
0,4665171,4665171,Process and intermediates for .beta.-lactam an...
1,4673737,4673737,7-acylamino-(or 7-amino)-3-trifluoromethylsulf...
2,4900548,4900548,Use of diethylcarbamazine to enhance antigen-a...
3,5317452,5317452,Aligning and attaching a lens to a source of e...
4,6042835,6042835,Prototype FelV isolates for use in disease mod...
5,6197515,6197515,Molecular recognition at surfaces derivatized ...
6,6211370,6211370,Asymmetric cycloaddition reactions
7,6248564,6248564,Mutant MHC class I molecules
8,7323346,7323346,Screening for gestational disorders
9,7772543,7772543,System and method for processing nanowires wit...


## Adding to the query other fields of interest

Above we were able to pull data with the default information on the patents (`patent_id`, `patent_number`, `patent_title`). 

It might be useful to know additional information on patents, such as patent classification and application date.

Let's look for those variables in the API Endpoint (http://www.patentsview.org/api/patent.html), and add those fields to our query.

We will use the USPC classification (United States Patent Classification) variable called `uspc_mainclass_title`.

The application date varible is called `app_date`.

To the URL created above, we will add the fields parameter: `&f=["patent_id", "patent_title","uspc_mainclass_title","app_date"]`.

In [84]:
url  # Above we defined a URL with the university name

'http://www.patentsview.org/api/patents/query?q={"assignee_organization":"stanford university"}'

Now let's add other fields.

In [85]:
url_fields = url + '&f=["patent_id", "patent_title","uspc_mainclass_title","app_date"]'

Let's check the URL.

In [86]:
url_fields

'http://www.patentsview.org/api/patents/query?q={"assignee_organization":"stanford university"}&f=["patent_id", "patent_title","uspc_mainclass_title","app_date"]'

In [87]:
r = requests.get(url_fields)  # Get response from the URL
r.status_code  # Check the status code

200

In [88]:
response = r.json()  # Convert response to JSON format

In [89]:
response  # View JSON

{'patents': [{'patent_id': '4200770',
   'patent_title': 'Cryptographic apparatus and method',
   'applications': [{'app_date': '1977-09-06', 'app_id': '05/830754'}],
   'uspcs': [{'uspc_mainclass_title': 'Cryptography'},
    {'uspc_mainclass_title': 'Electrical computers and digital processing systems:  support'}]},
  {'patent_id': '4214918',
   'patent_title': 'Method of forming polycrystalline semiconductor interconnections, resistors and contacts by applying radiation beam',
   'applications': [{'app_date': '1978-10-12', 'app_id': '05/950828'}],
   'uspcs': [{'uspc_mainclass_title': 'Metal treatment'},
    {'uspc_mainclass_title': 'Electric heating'},
    {'uspc_mainclass_title': 'Active solid-state devices (e.g., transistors, solid-state diodes)'},
    {'uspc_mainclass_title': 'Coating processes'},
    {'uspc_mainclass_title': 'Semiconductor device manufacturing: process'}]},
  {'patent_id': '4233671',
   'patent_title': 'Read only memory and integrated circuit and method of progr

For complex nested JSON files, let's try using the `json_normalize` function in `pandas` (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html).

In [90]:
"""If we just apply the function to the response, it will create a column for each key."""
json_normalize(response)  

,patents,count,total_patent_count
0,"[{'patent_id': '4200770', 'patent_title': 'Cry...",25,143


We want to break down the `patents` key further. Let's apply the function to `response['patents']`.

In [91]:
json_normalize(response['patents']).head()

,patent_id,patent_title,applications,uspcs
0,4200770,Cryptographic apparatus and method,"[{'app_date': '1977-09-06', 'app_id': '05/8307...","[{'uspc_mainclass_title': 'Cryptography'}, {'u..."
1,4214918,Method of forming polycrystalline semiconducto...,"[{'app_date': '1978-10-12', 'app_id': '05/9508...","[{'uspc_mainclass_title': 'Metal treatment'}, ..."
2,4233671,Read only memory and integrated circuit and me...,"[{'app_date': '1979-01-05', 'app_id': '06/0013...",[{'uspc_mainclass_title': 'Active solid-state ...
3,4234352,Thermophotovoltaic converter and cell for use ...,"[{'app_date': '1978-07-26', 'app_id': '05/9281...",[{'uspc_mainclass_title': 'Batteries: thermoe...
4,4325611,Electrochromic material and electro-optical di...,"[{'app_date': '1979-12-26', 'app_id': '06/1065...","[{'uspc_mainclass_title': 'Compositions'}, {'u..."


Much better. We can now see the `patent_id`, `patent_title`, but we still need to 
break down `applications` and `uspcs` further. For that, we will specify the `record_path`.

In [92]:
# First, specify the record path for the applications
json_normalize(response['patents'], record_path='applications').head()

,app_date,app_id
0,1977-09-06,05/830754
1,1978-10-12,05/950828
2,1979-01-05,06/001360
3,1978-07-26,05/928103
4,1979-12-26,06/106547


It shows us the values of the `applications` key, but we also want to know the associated `patent_id` and `patent_title`. We can specify those variables using the `meta` argument.

In [93]:
json_normalize(response['patents'], record_path='applications', meta=['patent_id','patent_title']).head()

,app_date,app_id,patent_id,patent_title
0,1977-09-06,05/830754,4200770,Cryptographic apparatus and method
1,1978-10-12,05/950828,4214918,Method of forming polycrystalline semiconducto...
2,1979-01-05,06/001360,4233671,Read only memory and integrated circuit and me...
3,1978-07-26,05/928103,4234352,Thermophotovoltaic converter and cell for use ...
4,1979-12-26,06/106547,4325611,Electrochromic material and electro-optical di...


This looks good. Let's save these results to a dataframe.

In [94]:
patent_app_date = json_normalize(response['patents'], record_path='applications', meta=['patent_id','patent_title'])

In [95]:
patent_app_date.head()

,app_date,app_id,patent_id,patent_title
0,1977-09-06,05/830754,4200770,Cryptographic apparatus and method
1,1978-10-12,05/950828,4214918,Method of forming polycrystalline semiconducto...
2,1979-01-05,06/001360,4233671,Read only memory and integrated circuit and me...
3,1978-07-26,05/928103,4234352,Thermophotovoltaic converter and cell for use ...
4,1979-12-26,06/106547,4325611,Electrochromic material and electro-optical di...


Now let's repeat the process for the `uspcs` key and save it into another dataframe.

In [96]:
json_normalize(response['patents'], record_path='uspcs', meta=['patent_id','patent_title']).head()

,uspc_mainclass_title,patent_id,patent_title
0,Cryptography,4200770,Cryptographic apparatus and method
1,Electrical computers and digital processing sy...,4200770,Cryptographic apparatus and method
2,Metal treatment,4214918,Method of forming polycrystalline semiconducto...
3,Electric heating,4214918,Method of forming polycrystalline semiconducto...
4,"Active solid-state devices (e.g., transistors,...",4214918,Method of forming polycrystalline semiconducto...


In [97]:
patent_class = json_normalize(response['patents'], record_path='uspcs', meta=['patent_id','patent_title'])

Now we can merge the two tables on `patent_id` and `patent_title` to get a complete table with the application date and the patent class.

In [98]:
df = patent_class.merge(patent_app_date,on=['patent_id','patent_title'])

In [99]:
df

,uspc_mainclass_title,patent_id,patent_title,app_date,app_id
0,Cryptography,4200770,Cryptographic apparatus and method,1977-09-06,05/830754
1,Electrical computers and digital processing sy...,4200770,Cryptographic apparatus and method,1977-09-06,05/830754
2,Metal treatment,4214918,Method of forming polycrystalline semiconducto...,1978-10-12,05/950828
3,Electric heating,4214918,Method of forming polycrystalline semiconducto...,1978-10-12,05/950828
4,"Active solid-state devices (e.g., transistors,...",4214918,Method of forming polycrystalline semiconducto...,1978-10-12,05/950828
5,Coating processes,4214918,Method of forming polycrystalline semiconducto...,1978-10-12,05/950828
6,Semiconductor device manufacturing: process,4214918,Method of forming polycrystalline semiconducto...,1978-10-12,05/950828
7,"Active solid-state devices (e.g., transistors,...",4233671,Read only memory and integrated circuit and me...,1979-01-05,06/001360
8,Miscellaneous active electrical nonlinear devi...,4233671,Read only memory and integrated circuit and me...,1979-01-05,06/001360
9,Static information storage and retrieval,4233671,Read only memory and integrated circuit and me...,1979-01-05,06/001360


### <span style="color:red">Checkpoint 2: Add other fields</span>

Try adding other fields of interest. Go to the Patents Endpoint (http://www.patentsview.org/api/patent.html) and pick other 2 fields of interest to add to the query and get the results.

In [100]:
url_fields = url + '&f=["patent_id","patent_title","assignee_city","assignee_state","assignee_country","assignee_first_name","assignee_last_name","app_date"]'

In [101]:
r = requests.get(url_fields)  # Get response from the URL
r.status_code  # Check the status code

200

In [102]:
response = r.json()  # Convert response to JSON format

In [103]:
json_normalize(response) 

,patents,count,total_patent_count
0,"[{'patent_id': '4200770', 'patent_title': 'Cry...",25,143


In [104]:
json_normalize(response['patents']).head()

,patent_id,patent_title,assignees,applications
0,4200770,Cryptographic apparatus and method,"[{'assignee_city': 'Palo Alto', 'assignee_stat...","[{'app_date': '1977-09-06', 'app_id': '05/8307..."
1,4214918,Method of forming polycrystalline semiconducto...,"[{'assignee_city': 'Stanford', 'assignee_state...","[{'app_date': '1978-10-12', 'app_id': '05/9508..."
2,4233671,Read only memory and integrated circuit and me...,"[{'assignee_city': 'Stanford', 'assignee_state...","[{'app_date': '1979-01-05', 'app_id': '06/0013..."
3,4234352,Thermophotovoltaic converter and cell for use ...,"[{'assignee_city': 'Stanford', 'assignee_state...","[{'app_date': '1978-07-26', 'app_id': '05/9281..."
4,4325611,Electrochromic material and electro-optical di...,"[{'assignee_city': 'Stanford', 'assignee_state...","[{'app_date': '1979-12-26', 'app_id': '06/1065..."


In [105]:
patent_assignees = json_normalize(response['patents'], record_path='assignees', meta=['patent_id','patent_title'])
patent_assignees.head()

,assignee_city,assignee_state,assignee_country,assignee_first_name,assignee_last_name,assignee_key_id,patent_id,patent_title
0,Palo Alto,CA,US,None,None,32068,4200770,Cryptographic apparatus and method
1,Stanford,CA,US,None,None,32068,4214918,Method of forming polycrystalline semiconducto...
2,Stanford,CA,US,None,None,32068,4233671,Read only memory and integrated circuit and me...
3,Stanford,CA,US,None,None,32068,4234352,Thermophotovoltaic converter and cell for use ...
4,Palo Alto,CA,US,None,None,349585,4234352,Thermophotovoltaic converter and cell for use ...


In [106]:
patent_applications = json_normalize(response['patents'], record_path='applications', meta=['patent_id','patent_title'])
patent_applications.head()

,app_date,app_id,patent_id,patent_title
0,1977-09-06,05/830754,4200770,Cryptographic apparatus and method
1,1978-10-12,05/950828,4214918,Method of forming polycrystalline semiconducto...
2,1979-01-05,06/001360,4233671,Read only memory and integrated circuit and me...
3,1978-07-26,05/928103,4234352,Thermophotovoltaic converter and cell for use ...
4,1979-12-26,06/106547,4325611,Electrochromic material and electro-optical di...


In [ ]:
df = patent_class.merge(patent_app_date,on=['patent_id','patent_title'])
df

## Customize the number of results

As you have noticed, by default, only 25 results are returned. To change the number of results returned (for example, 50 results), add the option parameter to the query URL: `&o={"page":1,"per_page":50}`. To get the second page, specify `&o={"page":2,"per_page":50)`.

Let's check the URL defined above.

In [ ]:
url_fields

In [ ]:
url_page_1 = url_fields + '&o={"page":1,"per_page":50}'

In [ ]:
r = requests.get(url_page_1)
r.status_code

In [ ]:
response_page_1 = r.json()  # Convert response to JSON format

Now the JSON shows 50 results (as noted in the variable `count`).

In [ ]:
response_page_1['count']

### <span style="color:red">Checkpoint 3: Customize the number of results</span>

Try customizing the number of returned results using the options parameter. 

**Note**: limit the number of results to no more than 100 during the in-class session, to avoid a heavy simultaneous use of the API (so the queries can run faster).

In [ ]:
# your code here...




## Optional

Please feel free to explore and practice all available options in the API Query Language section of the PatentsView website (http://www.patentsview.org/api/query-language.html).